## Tutorial: Using the generic interface to access model attributes

`COBREXA` makes use of a generic interface that enforces a uniform set of accessor functions for model attributes. This approach ensures that any custom function you create and test using, e.g. `JSONModel`s, also works on, e.g. `SBMLModel`s, `MATModel`s, `StandardModel`s, `CoreModel`s, and `CoreModelCoupled`. 

### Load the example models
As before, we are going to use the *E. coli* toy model to demonstrate the capabilities of `COBREXA`.

In [2]:
# Download the model files if they don't already exist
!isfile("e_coli_core.mat") && download("http://bigg.ucsd.edu/static/models/e_coli_core.mat", "e_coli_core.mat")
!isfile("e_coli_core.json") && download("http://bigg.ucsd.edu/static/models/e_coli_core.json", "e_coli_core.json")
!isfile("e_coli_core.xml") &&  download("http://bigg.ucsd.edu/static/models/e_coli_core.xml", "e_coli_core.xml")

using COBREXA

### Examples of accessor functions

Here is a list of all the accessor functions:

In [68]:
for (i, method) in enumerate(filter(x -> endswith(string(x.file), "MetabolicModel.jl"), methodswith(MetabolicModel, COBREXA)))
    println(i,") ", method.name)
end

1) balance
2) bounds
3) coupling
4) coupling_bounds
5) gene_annotations
6) gene_notes
7) genes
8) metabolite_annotations
9) metabolite_charge
10) metabolite_compartment
11) metabolite_formula
12) metabolite_notes
13) metabolites
14) n_coupling_constraints
15) n_genes
16) n_metabolites
17) n_reactions
18) objective
19) reaction_annotations
20) reaction_gene_association
21) reaction_notes
22) reaction_subsystem
23) reactions
24) stoichiometry


Refer to the function documentation in the `COBREXA` docs for more information. 

The documentation of each accessor function is easily accessed using the normal Julia `?` command followed by the function name. For example:

In [69]:
?stoichiometry

search: stoichiometry



```
stoichiometry(a::MetabolicModel)::SparseMat
```

Get the sparse stoichiometry matrix of a model.

---

```
stoichiometry(a::CoreModel)::SparseMat
```

`CoreModel` stoichiometry matrix.

---

```
stoichiometry(model::JSONModel)
```

Get the stoichiometry. Assuming the information is stored in reaction object under key `.metabolites`.

---

```
stoichiometry(m::MATModel)
```

Extract the stoichiometry matrix, stored under key `S`.

---

```
stoichiometry(a::SBMLModel)::SparseMat
```

Recreate the stoichiometry matrix from the SBML model.

---

```
stoichiometry(model::StandardModel)
```

Return the stoichiometric matrix associated with `model` in sparse format.


### Using the accessor functions

The generic interface makes it easy to access model attributes or implement your own functions that work with any model type, should you desire to do so. 

Let's load all the model types to demonstrate.

In [71]:
# load all the models! 
json_model = load_model("e_coli_core.json")
mat_model = load_model("e_coli_core.mat")
sbml_model = load_model("e_coli_core.json")
std_model = load_model(StandardModel, "e_coli_core.mat")
core_model = load_model(CoreModel, "e_coli_core.mat"); # TODO: remove ; when core coupled is fixed
# core_coupled_model = load_model(CoreModelCoupled, "e_coli_core.mat"); # broken..

Suppose you wanted to know which reactions are present in your model, this can easily be achieved by `reactions`:

In [72]:
rxn_ids = reactions(json_model)

95-element Vector{String}:
 "PFK"
 "PFL"
 "PGI"
 "PGK"
 "PGL"
 "ACALD"
 "AKGt2r"
 "PGM"
 "PIt2r"
 "ALCD2x"
 "ACALDt"
 "ACKr"
 "PPC"
 ⋮
 "ICL"
 "LDH_D"
 "MALS"
 "MALt2_2"
 "MDH"
 "ME1"
 "ME2"
 "NADH16"
 "NADTRHD"
 "NH4t"
 "O2t"
 "PDH"

Moreover, the generic interface is the same for each model type, so you would call the exact same function on any `COBREXA` model and get the same list of reactions (if the underlying model is the same). Note, the order may not be the same.

In [76]:
issetequal(reactions(json_model), reactions(sbml_model)) && 
issetequal(reactions(json_model), reactions(mat_model)) && 
issetequal(reactions(json_model), reactions(std_model)) && 
issetequal(reactions(json_model), reactions(core_model)) # TODO: add coupled model once it is fixed.

true

This generic interface is makes working with different models really easy because you don't have to change *anything* about your analysis pipeline, you call the exact same functions, no matter what model type you are working with! 